In [12]:
from kafka import KafkaProducer
import threading
import json
import random
from time import sleep

from equipo.random_tweets import generate_random_tweets  

KAFKA_SERVER = 'fdb4e6c32fcb:9093'
TOPIC_PREFIX = 'tweet-'

In [13]:
def worker(user_id: int, topic: str):

    num_messages = random.randint(300, 500)

    # Initialize the Kafka producer
    producer = KafkaProducer(
        bootstrap_servers=KAFKA_SERVER,
        value_serializer=lambda v: json.dumps(v).encode('utf-8')  # serialize data as JSON
    )

    print(f"[Thread-{user_id}] Iniciando, topic={topic}, mensajes={num_messages}")
    
    # Produce data to Kafka topic
    try:
        for tweet in generate_random_tweets(num_messages, user_id):
            # Send data to Kafka
            producer.send(topic, tweet)
            print(f"[Thread-{user_id}] Sent to {topic}: {tweet['tweet_id']}")

            sleep(1.5)

    except KeyboardInterrupt:
        print(f"[Thread-{user_id}] Interrupted.")
        
    finally:
        producer.close()
        print(f"[Thread-{user_id}] Producer closed.")

In [ ]:
if __name__ == "__main__":
    threads = []

    for user_id in range(1, 5):
        topic = f"{TOPIC_PREFIX}{user_id}"
        t = threading.Thread(
            target=worker,
            args=(user_id, topic),
            name=f"Thread-{user_id}"
        )
        threads.append(t)
        t.start()

    # Espera a que todos los hilos terminen
    for t in threads:
        t.join()

    print("Finished.")


[Thread-11] Iniciando, topic=tweet-11, mensajes=378
[Thread-12] Iniciando, topic=tweet-12, mensajes=400
[Thread-10] Iniciando, topic=tweet-10, mensajes=372
[Thread-13] Iniciando, topic=tweet-13, mensajes=362
[Thread-22] Iniciando, topic=tweet-22, mensajes=319
[Thread-14] Iniciando, topic=tweet-14, mensajes=462
[Thread-16] Iniciando, topic=tweet-16, mensajes=392
[Thread-19] Iniciando, topic=tweet-19, mensajes=341
[Thread-23] Iniciando, topic=tweet-23, mensajes=424
[Thread-18] Iniciando, topic=tweet-18, mensajes=317
[Thread-20] Iniciando, topic=tweet-20, mensajes=409
[Thread-21] Iniciando, topic=tweet-21, mensajes=344
[Thread-17] Iniciando, topic=tweet-17, mensajes=490
[Thread-10] Sent to tweet-10: 1bbfb6d2-9239-43eb-84de-5270f421aca1
[Thread-15] Iniciando, topic=tweet-15, mensajes=368
[Thread-24] Iniciando, topic=tweet-24, mensajes=427
[Thread-26] Iniciando, topic=tweet-26, mensajes=338
[Thread-12] Sent to tweet-12: 3a7861fa-90ef-4d2b-9eab-33c7b6ee7ccc
[Thread-28] Iniciando, topic=tweet